In [146]:
import pandas as pd
import numpy as np
import requests
import json
from pandas.io.json import json_normalize
from ratelimit import limits, sleep_and_retry
import requests

In [118]:
#pip install ratelimit

In [152]:
two_minutes = 120


@sleep_and_retry
@limits(calls=90, period=two_minutes)
def call_api(url):
    response = requests.get(url)

    if response.status_code != 200:
        raise Exception('API response: {}'.format(response.status_code))
    return response


In [65]:
# Global Variables
KEY = 'RGAPI-e8ab8c77-0e55-4562-814f-f7e64301e8d0'
API_BASE = 'https://na1.api.riotgames.com/'
API_BASE_REGION = 'https://americas.api.riotgames.com/'
 
TIERS = ['IRON' , 'BRONZE', 'SILVER', 'GOLD', 'PLATINUM', 'DIAMOND']
DIVISIONS = ['I'] # , 'II', 'III', 'IV']
TIERS_LIST = []
for tier in TIERS:
  for division in DIVISIONS:
    TIER_DIVISION = tier + '/' + division
    TIERS_LIST.append(TIER_DIVISION)

TIER_URLS = []
for tiers_divis in TIERS_LIST: 
  url = API_BASE + 'tft/league/v1/entries/' + tiers_divis + '?page=1&api_key=' + KEY
  TIER_URLS.append(url)

In [66]:
TIER_URLS

['https://na1.api.riotgames.com/tft/league/v1/entries/IRON/I?page=1&api_key=RGAPI-e8ab8c77-0e55-4562-814f-f7e64301e8d0',
 'https://na1.api.riotgames.com/tft/league/v1/entries/BRONZE/I?page=1&api_key=RGAPI-e8ab8c77-0e55-4562-814f-f7e64301e8d0',
 'https://na1.api.riotgames.com/tft/league/v1/entries/SILVER/I?page=1&api_key=RGAPI-e8ab8c77-0e55-4562-814f-f7e64301e8d0',
 'https://na1.api.riotgames.com/tft/league/v1/entries/GOLD/I?page=1&api_key=RGAPI-e8ab8c77-0e55-4562-814f-f7e64301e8d0',
 'https://na1.api.riotgames.com/tft/league/v1/entries/PLATINUM/I?page=1&api_key=RGAPI-e8ab8c77-0e55-4562-814f-f7e64301e8d0',
 'https://na1.api.riotgames.com/tft/league/v1/entries/DIAMOND/I?page=1&api_key=RGAPI-e8ab8c77-0e55-4562-814f-f7e64301e8d0']

In [120]:
# Get list of each summoner in each tier
tier_frames = []

for url in TIER_URLS:
  REQUEST_TIERS = call_api(url)
  tier_data = json.loads(REQUEST_TIERS.text)
  tier_df = json_normalize(tier_data)
  tier_frames.append(tier_df)
divisions = pd.concat(tier_frames, ignore_index=True, sort=True)

In [121]:
divisions['division_tier'] = divisions['tier'] + '_' + divisions['rank'] 
divisions

,freshBlood,hotStreak,inactive,leagueId,leaguePoints,losses,queueType,rank,summonerId,summonerName,tier,veteran,wins,division_tier
0,False,False,False,bba31c20-0f0a-11ea-9d15-0292af83ade6,94,5,RANKED_TFT,I,yuST92qI7G8ZiMWVBzHJ6e2xONnD6Rc1R-AfC3IU-FEsK8xY,Thesmokingpun,IRON,False,0,IRON_I
1,False,False,False,38ef74e0-2049-11ea-af12-e2e7dba1695f,41,22,RANKED_TFT,I,cClZh8jH4gTqTlFoz_fYFO6OQwLvWEvyJaCx515J_g6Ml3GX,TwitchDrDescend,IRON,False,0,IRON_I
2,False,False,False,3e06330f-c4f3-4a08-a497-e81f6df10588,13,9,RANKED_TFT,I,8VN_E8UqdztjMGB_OU5taldKe8rYdKqbQeQNJij7aa8Bx8BS,Iskoog,IRON,False,0,IRON_I
3,False,False,False,f4694230-0bb0-11ea-b9fc-eeb4098ebaa0,11,6,RANKED_TFT,I,Ild9nWTxkyF8XWYHIbSKkO4l_HuWKnc-oCBhnMwm2txdJVdu,paybackisthename,IRON,False,0,IRON_I
4,False,False,False,13e67db0-19e1-11ea-8ce1-0256c6254f55,30,5,RANKED_TFT,I,cID8eSPpX2QVsydzTIdzHtXD1GBIjf_kTsl3zIXIctjPOspl,Dasheng Tian,IRON,False,2,IRON_I
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1225,False,False,False,4dbb9270-3104-11ea-b978-da957a0ba512,0,246,RANKED_TFT,I,GKcNbHuCmrv7ctx1PsyX5GUWt2TWG_ky8j9Ub8mEE5PFXRQZ,h3li0n,DIAMOND,False,63,DIAMOND_I
1226,False,False,False,d3354c0b-1e5c-412a-9d41-7b9e63892ee9,25,383,RANKED_TFT,I,MUibO7QmaPCwU9TXAAag32IlZVXBoQ-9AizuzFNLiEqSYKNB,1000water,DIAMOND,False,78,DIAMOND_I
1227,False,False,False,81c15780-1b65-11ea-9a33-362bc5ddecfc,50,235,RANKED_TFT,I,IrMF-MpkB_aiAMNARMGKZ9NK3QEtQmUUHklOfRoeJ1uddWT6,twitch tv guffee,DIAMOND,False,47,DIAMOND_I
1228,False,False,False,3577d755-3aae-4ee1-8301-7e4dccdb25fc,31,89,RANKED_TFT,I,Tv8hZTghDbMH2jxS77fD5jVxh9jlmy4W-6mt82lCUOiGnqVr,theruggedman420,DIAMOND,False,18,DIAMOND_I


In [122]:
divisions

,freshBlood,hotStreak,inactive,leagueId,leaguePoints,losses,queueType,rank,summonerId,summonerName,tier,veteran,wins,division_tier
0,False,False,False,bba31c20-0f0a-11ea-9d15-0292af83ade6,94,5,RANKED_TFT,I,yuST92qI7G8ZiMWVBzHJ6e2xONnD6Rc1R-AfC3IU-FEsK8xY,Thesmokingpun,IRON,False,0,IRON_I
1,False,False,False,38ef74e0-2049-11ea-af12-e2e7dba1695f,41,22,RANKED_TFT,I,cClZh8jH4gTqTlFoz_fYFO6OQwLvWEvyJaCx515J_g6Ml3GX,TwitchDrDescend,IRON,False,0,IRON_I
2,False,False,False,3e06330f-c4f3-4a08-a497-e81f6df10588,13,9,RANKED_TFT,I,8VN_E8UqdztjMGB_OU5taldKe8rYdKqbQeQNJij7aa8Bx8BS,Iskoog,IRON,False,0,IRON_I
3,False,False,False,f4694230-0bb0-11ea-b9fc-eeb4098ebaa0,11,6,RANKED_TFT,I,Ild9nWTxkyF8XWYHIbSKkO4l_HuWKnc-oCBhnMwm2txdJVdu,paybackisthename,IRON,False,0,IRON_I
4,False,False,False,13e67db0-19e1-11ea-8ce1-0256c6254f55,30,5,RANKED_TFT,I,cID8eSPpX2QVsydzTIdzHtXD1GBIjf_kTsl3zIXIctjPOspl,Dasheng Tian,IRON,False,2,IRON_I
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1225,False,False,False,4dbb9270-3104-11ea-b978-da957a0ba512,0,246,RANKED_TFT,I,GKcNbHuCmrv7ctx1PsyX5GUWt2TWG_ky8j9Ub8mEE5PFXRQZ,h3li0n,DIAMOND,False,63,DIAMOND_I
1226,False,False,False,d3354c0b-1e5c-412a-9d41-7b9e63892ee9,25,383,RANKED_TFT,I,MUibO7QmaPCwU9TXAAag32IlZVXBoQ-9AizuzFNLiEqSYKNB,1000water,DIAMOND,False,78,DIAMOND_I
1227,False,False,False,81c15780-1b65-11ea-9a33-362bc5ddecfc,50,235,RANKED_TFT,I,IrMF-MpkB_aiAMNARMGKZ9NK3QEtQmUUHklOfRoeJ1uddWT6,twitch tv guffee,DIAMOND,False,47,DIAMOND_I
1228,False,False,False,3577d755-3aae-4ee1-8301-7e4dccdb25fc,31,89,RANKED_TFT,I,Tv8hZTghDbMH2jxS77fD5jVxh9jlmy4W-6mt82lCUOiGnqVr,theruggedman420,DIAMOND,False,18,DIAMOND_I


In [99]:
# Get summoner name from divisions data frame / keeping it at 5 for now for testing
summoner_names = divisions.loc[ :, 'summonerName' ]

In [100]:
summoner_names

0          Thesmokingpun
1        TwitchDrDescend
2                 Iskoog
3       paybackisthename
4           Dasheng Tian
              ...       
1225              Igoaff
1226              h3li0n
1227           1000water
1228    twitch tv guffee
1229     theruggedman420
Name: summonerName, Length: 1230, dtype: object

In [125]:
summoner_frames = []

for summoner in summoner_names:
  summoner_url = API_BASE + 'tft/summoner/v1/summoners/by-name/' + summoner + '?api_key=' + KEY
  REQUEST_NAME = call_api(summoner_url)
  summoner_data = json.loads(REQUEST_NAME.text)
  summoner_df = json_normalize(summoner_data)
  summoner_frames.append(summoner_df)
join_summoners = pd.concat(summoner_frames, ignore_index=True, sort=True)

RateLimitException: too many calls

In [126]:
join_summoners = join_summoners.merge(divisions[['summonerName','division_tier']],left_on='name', right_on = 'summonerName')

In [127]:
join_summoners.head()

,accountId,id,name,profileIconId,puuid,revisionDate,status.message,status.status_code,summonerLevel,summonerName_x,division_tier_x,summonerName_y,division_tier_y
0,zI283lT_AmT46PYA4ryvDu6_pCEQWelYXsDEFnkVuY46eK...,yuST92qI7G8ZiMWVBzHJ6e2xONnD6Rc1R-AfC3IU-FEsK8xY,Thesmokingpun,4080.0,Ki1KO62ygcycSVm6LoqHuxrL2CMzEF-bU2uMKrms9pPUog...,1.577783e+12,NaN,NaN,174.0,Thesmokingpun,IRON_I,Thesmokingpun,IRON_I
1,HoX-vMezy19MFPStaMIWJkSJo7TqzKUiiZcdRLQJNiLNfj...,cClZh8jH4gTqTlFoz_fYFO6OQwLvWEvyJaCx515J_g6Ml3GX,TwitchDrDescend,1437.0,Qo8MH3V2TR24SNwj15PHwdLnmls1zV2TshFuMzLrwoK30-...,1.583209e+12,NaN,NaN,27.0,TwitchDrDescend,IRON_I,TwitchDrDescend,IRON_I
2,JpUFYXoEICpGy_pc_bpIF834PYUkAxCZU_hNvNPs9Zedy4...,8VN_E8UqdztjMGB_OU5taldKe8rYdKqbQeQNJij7aa8Bx8BS,Iskoog,3791.0,rOLjK2bI7u1Vgf0UW6N57SMSE26R_-Q5sNj5CCYLNPhzCu...,1.583265e+12,NaN,NaN,38.0,Iskoog,IRON_I,Iskoog,IRON_I
3,L1syM0T2vG7fFnguMokQk2F_XmRwby3YtItKvx-naNEIFp...,Ild9nWTxkyF8XWYHIbSKkO4l_HuWKnc-oCBhnMwm2txdJVdu,paybackisthename,4088.0,PhT-sWyWBFP7e9ks7BgJ-jJ4tbI1Pu9g_YqQWf4v-pWQJb...,1.583111e+12,NaN,NaN,147.0,paybackisthename,IRON_I,paybackisthename,IRON_I
4,A70edyG8emCOgQguKKxXH_UCI1Uw2ypDg7Ogai9khr4sHj...,cID8eSPpX2QVsydzTIdzHtXD1GBIjf_kTsl3zIXIctjPOspl,Dasheng Tian,3546.0,NaxTv335VOuWqE47Re-qtlj6IFINYFe9mO0pH_lYndCdlF...,1.580410e+12,NaN,NaN,13.0,Dasheng Tian,IRON_I,Dasheng Tian,IRON_I


In [128]:
# Get puuid from summoners data frame
summoner_puuid = join_summoners.loc[ : , 'puuid' ]

In [129]:
summoner_puuid

0      Ki1KO62ygcycSVm6LoqHuxrL2CMzEF-bU2uMKrms9pPUog...
1      Qo8MH3V2TR24SNwj15PHwdLnmls1zV2TshFuMzLrwoK30-...
2      rOLjK2bI7u1Vgf0UW6N57SMSE26R_-Q5sNj5CCYLNPhzCu...
3      PhT-sWyWBFP7e9ks7BgJ-jJ4tbI1Pu9g_YqQWf4v-pWQJb...
4      NaxTv335VOuWqE47Re-qtlj6IFINYFe9mO0pH_lYndCdlF...
                             ...                        
483    vGSl1tTNCooor5w-JONvMkmGB2weZY90sOvzYeanUksSap...
484    tSNysfUVTQpkGd6g-9OyltdRpV7uV4obD_sCc_O7oT8Ga_...
485    mbGY-ForOVQbKvc35LsLYsOCQ39rbOLyR2wi8VsZSvM_fr...
486    HKfngh-LDTZFEEaxKB-3SLq0UHHWxWJzZ_EwinK3LHoobJ...
487    pv5zRB3_PjGeTo_GS22_W2lSz_TJdawcBACUKFx6jCHGfy...
Name: puuid, Length: 488, dtype: object

In [153]:
game_id_frames = []
match_count = '1'

for puuid in summoner_puuid:
  match_url = API_BASE_REGION + '/tft/match/v1/matches/by-puuid/' + puuid + '/ids?count=' + match_count + '&api_key=' + KEY
  REQUEST_MATCH = call_api(match_url)
  match_data = json.loads(REQUEST_MATCH.text)
  match_df = pd.Series(match_data)
  game_id_frames.append(match_df)
game_id_data = pd.concat(game_id_frames, ignore_index=True, sort=True)

In [154]:
# need to convert to series
game_id_list = game_id_data.loc[:]

In [155]:
game_id_list

0      NA1_3220227240
1      NA1_3313188067
2      NA1_3302506161
3      NA1_3313864862
4      NA1_3281128705
            ...      
483    NA1_3282265169
484    NA1_3275896562
485    NA1_3299509542
486    NA1_3312509236
487    NA1_3293994253
Length: 488, dtype: object

In [156]:
match_info_frames = []
match_count = '2'

for game_id in game_id_list:
  result_url = API_BASE_REGION + 'tft/match/v1/matches/' + game_id + '?api_key=' + KEY
  REQUEST_RESULT = call_api(result_url)
  result_data = json.loads(REQUEST_RESULT.text)
  result_df = json_normalize(result_data)
  match_info_frames.append(result_df)
match_info_data = pd.concat(match_info_frames, ignore_index=True, sort=True)

In [157]:
match_info_data

,info.game_datetime,info.game_length,info.game_version,info.participants,info.queue_id,info.tft_set_number,metadata.data_version,metadata.match_id,metadata.participants
0,1574977840977,2028.678955,Version 9.23.299.3089 (Nov 25 2019/17:12:47) [...,[{'companion': {'content_ID': 'b5cab758-7194-4...,1100,2,3,NA1_3220227240,[WwAaD7WefPc_IIK298IpwEWE-FYn2s3kpLM9kSdsaxGTA...
1,1583209488712,1885.422363,Version 10.4.308.9400 (Feb 14 2020/17:25:03) [...,[{'companion': {'content_ID': '8c27f67c-90b3-4...,1100,2,4,NA1_3313188067,[DPM1MPKwrBtNsXNARpLMQpQ9XPkspQgFuOdw1qhu8W5F3...
2,1582241300313,1823.497559,Version 10.4.308.9400 (Feb 14 2020/17:25:03) [...,[{'companion': {'content_ID': 'c4508a6f-123d-4...,1100,2,4,NA1_3302506161,[2K2P4x46UgHOhGTI32nxjcT_PahCXUisXEowi6vbatO0O...
3,1583269972996,1979.711426,Version 10.4.308.9400 (Feb 14 2020/17:25:03) [...,[{'companion': {'content_ID': 'd07d0b76-cd0d-4...,1100,2,4,NA1_3313864862,[plODai2a7lalVCepZefOt1MIkaJzkKAHCzb3hqbXVJa0B...
4,1580409699694,2011.223267,Version 10.2.305.4739 (Jan 21 2020/15:48:11) [...,[{'companion': {'content_ID': '5897ad9f-4665-4...,1100,2,4,NA1_3281128705,[NaxTv335VOuWqE47Re-qtlj6IFINYFe9mO0pH_lYndCdl...
...,...,...,...,...,...,...,...,...,...
483,1580522180962,2025.713257,Version 10.2.305.4739 (Jan 21 2020/15:48:11) [...,[{'companion': {'content_ID': '270079d1-2d24-4...,1100,2,4,NA1_3282265169,[RFvp4LBKrv9veGL4X5K7wlWc1hmYlFKookp5FZQPTxfxc...
484,1579981259673,2061.330322,Version 10.2.305.4739 (Jan 21 2020/15:48:11) [...,[{'companion': {'content_ID': 'f56ecac7-ed16-4...,1100,2,4,NA1_3275896562,[Xtx4PAkoy9yg63hkwYT8uKGJf2Rfm3-V-jWUHdmHVKQIw...
485,1581977342583,1839.294800,Version 10.3.307.7898 (Feb 06 2020/20:39:51) [...,[{'companion': {'content_ID': 'eb78f626-c11a-4...,1100,2,4,NA1_3299509542,[08HtiyMArpgyyAatfz4WOtfINeYqKd4qmOFjjgobrwq0K...
486,1583125483751,2077.494873,Version 10.4.308.9400 (Feb 14 2020/17:25:03) [...,[{'companion': {'content_ID': 'b36d46ca-120e-4...,1100,2,4,NA1_3312509236,[EvKg6iidQntOdNH57JywDYAW7gr4ucxHPw6GmVc1k65Mq...


In [158]:
match_info_data.columns

Index(['info.game_datetime', 'info.game_length', 'info.game_version',
       'info.participants', 'info.queue_id', 'info.tft_set_number',
       'metadata.data_version', 'metadata.match_id', 'metadata.participants'],
      dtype='object')

In [159]:
pd.options.display.max_seq_items = 2000

In [160]:
match_info_data['info.participants'][2]

[{'companion': {'content_ID': 'c4508a6f-123d-443f-84e7-e40ea526cd7f',
   'skin_ID': 10,
   'species': 'PetPenguKnight'},
  'gold_left': 2,
  'last_round': 27,
  'level': 6,
  'placement': 7,
  'players_eliminated': 0,
  'puuid': '2K2P4x46UgHOhGTI32nxjcT_PahCXUisXEowi6vbatO0OKKH77lGgjPgUHu-wwS_AkcEuNsqbXlJtA',
  'time_eliminated': 1490.4603271484375,
  'total_damage_to_players': 42,
  'traits': [{'name': 'Berserker',
    'num_units': 1,
    'style': 0,
    'tier_current': 0,
    'tier_total': 2},
   {'name': 'Inferno',
    'num_units': 3,
    'style': 1,
    'tier_current': 1,
    'tier_total': 3},
   {'name': 'Light',
    'num_units': 1,
    'style': 0,
    'tier_current': 0,
    'tier_total': 3},
   {'name': 'Set2_Ranger',
    'num_units': 2,
    'style': 1,
    'tier_current': 1,
    'tier_total': 3},
   {'name': 'Shadow',
    'num_units': 3,
    'style': 1,
    'tier_current': 1,
    'tier_total': 2},
   {'name': 'Summoner',
    'num_units': 3,
    'style': 1,
    'tier_current': 1,

In [161]:
#df = pd.Series(data={match_info_data['info.participants'][0]})
# dct = {'A':5, 'B':4, 'C':-1}
# df['min_count'] = df['category'].map(dct)
# df = df.query('count>min_count')

participants_df = []
for num in range(len(match_info_data['info.participants'])):
    
    data = match_info_data['info.participants'][num]
    data_df = pd.DataFrame.from_dict(data)
    participants_df.append(data_df)
    game_df = pd.concat(participants_df, ignore_index=True, sort=True)


game_df


KeyboardInterrupt: 

In [ ]:
# REPLACE PUUID WITH NAME?

#name_ = game_df['puuid'] 
#for puuid in game_df['puuid']:
#game_df['puuid'] = join_summoners['name'].map(join_summoners.set_index('name')['puuid'])

In [ ]:
range(len(game_df['units']))

In [ ]:
game_df_test = []
#[game_df['unit_1_character_id'][row] for row in game_df['units'][row][0]['character_id']
# game_df['unit_1_items']
# game_df['unit_1_name']
# game_df['unit_1_rarity']
# game_df['unit_1_tier']
#game_df['unit_1_character_id'] = [game_df['units'][0]['character_id'] for row in game_df['units']]


In [ ]:
game_df

In [ ]:
traits_data = game_df['traits']
traits_df = pd.DataFrame.from_dict(traits_data)
traits_df['traits'][0]

In [ ]:
units_data = game_df['units']
units_df = pd.DataFrame.from_dict(units_data)
units_df['units'][1]

In [ ]:
# join the unit name with tier value in a list separated by comma
game_df['units_name'] = [','.join([unit['name']+'_'+str(unit['tier']) for unit in units]) for units in game_df['units']]

# create dummy columns of the list of 'unit_tier' per player and add it to original df
units_df = game_df['units_name'].str.get_dummies(sep = ',')

In [ ]:
# join the trait name with set tier value in a list separated by comma
game_df['trait_name'] = [','.join([trait['name']+'_'+str(trait['tier_current']) for trait in traits]) for traits in game_df['traits']]

# create dummy columns of the list of 'unit_tier' per player and add it to original df
traits_df = game_df['trait_name'].str.get_dummies(sep = ',')

In [ ]:
# join the dummy columns with original game_df
game_df = game_df.join(units_df)
game_df = game_df.join(traits_df)


In [ ]:
game_df = game_df.merge(join_summoners[['puuid','summonerName','division_tier']],left_on='puuid', right_on ='puuid')

In [ ]:
game_df.to_csv('./datasets/full_data.csv')

In [ ]:
game_df = pd.read_csv('./datasets/full_data.csv')

In [ ]:
game_df